In [1]:
# Importing the required libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../datasets/power-meter-nov-data.csv')
df.drop(columns=["_id", "id"]) # Drop Uncessary Columns

,meter,data.value,data.createdAt
0,11,341411552,2024-11-04T04:48:36.338Z
1,11,341411808,2024-11-04T04:49:12.311Z
2,11,341412064,2024-11-04T04:49:48.296Z
3,11,341412320,2024-11-04T04:50:24.279Z
4,11,341412544,2024-11-04T04:51:00.262Z
...,...,...,...
49539,11,341309440,2024-10-30T08:27:01.823Z
49540,11,341309440,2024-10-30T08:27:37.779Z
49541,11,341309440,2024-10-30T08:28:13.740Z
49542,11,341309440,2024-10-30T08:28:49.691Z
